# Fonction permettant de récupérer les films sur TMDB

## Chargement des libraires, de la clé API et délacration de la base URL

In [39]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import joblib

load_dotenv()
api_key = os.getenv("TMDB_API_KEY")

if not api_key:
    print("Erreur : clé API non trouvée.")
    exit()

base_url = "https://api.themoviedb.org/3"

df_movies = joblib.load("df_movies.pkl")

## Récupérer les correspondances des noms de genres et ids sur TMDB

In [40]:
def get_genre_mapping(api_key):
    load_dotenv()
    api_key = os.getenv("TMDB_API_KEY")

    if not api_key:
        print("Erreur : clé API non trouvée.")
        exit()

    url = f'{base_url}/genre/movie/list?api_key={api_key}&language=en-EN'
    response = requests.get(url)
    
    if response.status_code == 200:
        genres = response.json().get("genres", [])
        return {genre["id"]: genre["name"] for genre in genres}
    else:
        print("Erreur lors de la récupération des genres.")
        return {}

In [ ]:
genres_mapping = get_genre_mapping(api_key=api_key)
genres_mapping

## Fonction pour récupérer les films "Now Playing"(actuellement au cinéma) de TMDB

In [42]:
def get_now_playing_movies(pages=100): 
    movies = []
    for page in range(1, pages + 1):
        url = f"{base_url}/movie/now_playing?api_key={api_key}&language=fr-FR&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            movies.extend(data['results'])
        else:
            print(f" Erreur HTTP {response.status_code} sur la page {page}")
    return movies

In [ ]:
movies = get_now_playing_movies()
movies

## Fonction pour récupérer les détails dés équipes

In [45]:
# on récupère l'id du film
def get_movie_credits(movie_id):
    url = f"{base_url}/movie/{movie_id}/credits?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# on récupère les détails du film
def get_movie_details(movie_id):
    url = f"{base_url}/movie/{movie_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# on récupère tous les équipes puis on combine tout dans une base
def fetch_movies_with_credits():
    movies = get_now_playing_movies()
    movie_data = []

    for movie in movies:
        movie_id = movie["id"]
        
        details = get_movie_details(movie_id)
        if not details:
            print(f"Impossible de récupérer les détails pour {movie_id}")
            continue

        genre_ids = movie.get("genre_ids", [])
        genres = [genres_mapping.get(genre_id) for genre_id in genre_ids if genres_mapping.get(genre_id)]

        # récupérer les crédits du film
        credits = get_movie_credits(movie_id)
        if not credits:
            print(f"Impossible de récupérer les crédits pour le film {movie_id})")
            continue

        actors = []
        actors_rank = []
        for index, cast in enumerate(credits["cast"]):
            actors.append(cast["name"])
            actors_rank.append(index + 1)

        directors = []
        for crew in credits["crew"]:
            if crew["job"] == "Director":
                directors.append(crew["name"])

        writers = []
        for crew in credits["crew"]:
            if crew["job"] == "Writer":
                writers.append(crew["name"])

        producers = []
        for crew in credits["crew"]:
            if crew["job"] == "Producer":
                producers.append(crew["name"])

        cinematographers = []
        for crew in credits["crew"]:
            if crew["job"] in ["Cinematographer", "Director of Photography"]:
                cinematographers.append(crew["name"])

        editors = []
        for crew in credits["crew"]:
            if crew["job"] == "Editor":
                editors.append(crew["name"])

        # Ajouter les données dans la liste
        movie_data.append({
            "id": movie_id,
            "original_title": details["original_title"],
            "release_date": details["release_date"],
            "vote_average": details["vote_average"],
            "vote_count": details["vote_count"],
            "popularity": details["popularity"],
            "overview": details["overview"],
            "poster_path": details["poster_path"],
            "genres": ", ".join(genres),
            "actors": ", ".join(actors),
            "actors_rank": ", ".join(map(str, actors_rank)),
            "directors": ", ".join(directors),
            "writers": ", ".join(writers),
            "producers": ", ".join(producers),
            "cinematographers": ", ".join(cinematographers),
            "editors": ", ".join(editors),
        })

    return pd.DataFrame(movie_data)
    

In [46]:
movie_data = fetch_movies_with_credits()
movie_data

,id,original_title,release_date,vote_average,vote_count,popularity,overview,poster_path,genres,actors,actors_rank,directors,writers,producers,cinematographers,editors
0,1286773,The Metropolitan Opera: Fidelio,2025-03-15,0.0,0,22.796,Following a string of awe-inspiring Met perfor...,/9DU8GX7tiIZQQ15MY4b44Rtgnba.jpg,Music,"Lise Davidsen, Ying Fang, David Butt Philip, M...","1, 2, 3, 4, 5, 6, 7",Jürgen Flimm,Ludwig van Beethoven,,,
1,1315476,Paradiso,2025-03-15,0.0,0,22.708,"Inside a cold and rusty bunker, Irina and Geor...",/jN5fvKt25DtWVoPTRaEQgGmmMqd.jpg,"Drama, Horror","Lucía Zamora, Jorge Cañas, Martin Taskov, Adel...","1, 2, 3, 4",Ivan Dimitrov,"Fernando Huete, Ivan Dimitrov",Bea Guedeja-Marrón Gil,Ivan Dimitrov,Ivan Dimitrov
2,1325495,逃走,2025-03-15,0.0,0,22.677,Follows the final days of left-wing terrorist ...,/l8v8L2OC95tB4za2oex6wye4TwW.jpg,,"Kanji Furutachi, Rairu Sugita, Soran Tamoto, M...","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13",Masao Adachi,Masao Adachi,Sanshirô Kobayashi,Yutaka Yamazaki,Tomoko Hiruta
3,1340727,Afterwards,2025-03-15,0.0,0,22.645,A young man receives advice from his deceased ...,/nR5oqvfkWw94vVChRRiZRwEXhCL.jpg,"Horror, Romance","Ryan Cason, Whitney Ann Bates, Tim Knutsen, Sh...","1, 2, 3, 4, 5, 6, 7",Gibson Almgren,Gibson Almgren,Melaina Koulos,Nicholas Emmanuele,Jon Anderson
4,1359744,Myself Two Seconds to Cry,2025-03-15,0.0,0,22.614,The digital presence left by people I knew who...,/cnqztuI8rYP2XkGDvhMUmLbqnNe.jpg,Music,,,Erik Sutch,Erik Sutch,John Scharbach,Katie Dance,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1444160,The Dalai Lama's Gift,2025-02-22,0.0,0,1.083,"In 1981, a Buddhist monk teaching at the Unive...",None,Documentary,Tenzin Gyatso,1,Ed Bastian,,,,
1996,1443850,Unrecognized Character Detected,2025-02-22,0.0,0,1.083,Two alien organisms engage in a playful confli...,None,,,,,,,,
1997,1443849,Imagine the Audience Naked,2025-02-22,0.0,0,1.083,Funk band Grog Machine walk out into the spotl...,None,,,,Reuben Underwood,,,,
1998,1443103,Chucks,2025-02-22,0.0,0,1.083,A mother's wish to fulfil her son's dream of o...,None,,,,"Anum Ehtesham, Rafay Rockingson",,,,


In [57]:
df_movies = joblib.load("df_movies.pkl")
df_movies.rename(columns={"originalTitle": "original_title"}, inplace=True)
df_now_playing = movie_data[~movie_data["original_title"].isin(df_movies["original_title"])]

df_now_playing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1697 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1697 non-null   int64  
 1   original_title    1697 non-null   object 
 2   release_date      1697 non-null   object 
 3   vote_average      1697 non-null   float64
 4   vote_count        1697 non-null   int64  
 5   popularity        1697 non-null   float64
 6   overview          1697 non-null   object 
 7   poster_path       1435 non-null   object 
 8   genres            1697 non-null   object 
 9   actors            1697 non-null   object 
 10  actors_rank       1697 non-null   object 
 11  directors         1697 non-null   object 
 12  writers           1697 non-null   object 
 13  producers         1697 non-null   object 
 14  cinematographers  1697 non-null   object 
 15  editors           1697 non-null   object 
dtypes: float64(2), int64(2), object(12)
memory usag

In [ ]:
Index(['tconst', 'titleType', 'originalTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres', 'directors', 'writers', 'averageRating',
       'numVotes', 'periode', 'budget', 'id', 'original_language', 'overview',
       'popularity', 'poster_path', 'production_countries', 'revenue',
       'spoken_languages', 'vote_average', 'vote_count', 'actors_name',
       'actors_rank', 'directors_name', 'writers_name', 'producers_name',
       'cinematographers_name', 'editors_name', 'genres_liste',
       'genre_History', 'genre_War', 'genre_Western', 'genre_Animation',
       'genre_Fantasy', 'genre_Adventure', 'genre_Biography', 'genre_News',
       'genre_Talk-Show', 'genre_Comedy', 'genre_Reality-TV', 'genre_Crime',
       'genre_Family', 'genre_Sport', 'genre_Action', 'genre_Sci-Fi',
       'genre_Drama', 'genre_Thriller', 'genre_Adult', 'genre_Mystery',
       'genre_Horror', 'genre_Romance', 'genre_Documentary', 'genre_Music',
       'genre_Film-Noir', 'genre_Unknown', 'genre_Game-Show', 'genre_Musical'],
      dtype='object')